# Data Preparation - IT2 - Choosing |best techniques per step in the flow 

In [ ]:
import cv2
import numpy as np
import pandas as pd
import random
import os
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm


In [ ]:
# Define the path to the folder containing the images to be processed
folder_path = '../data/subset'  # Update this path to point to your specific folder containing images

## Loading Images and stats

In [ ]:
def load_images_from_folder(folder_path, extensions=('.png', '.jpg', '.jpeg', '.JPG')):
    """
    Load all image file paths from a specified folder that match the given file extensions.

    Parameters:
    folder_path (str): The path to the folder containing the images.
    extensions (tuple of str): A tuple of file extensions to filter the images by. 
                               Default is ('.png', '.jpg', '.jpeg', '.JPG').

    Returns:
    list: A list of full file paths to images in the folder that match the specified extensions.
    
    Raises:
    FileNotFoundError: If the specified folder does not exist.
    """

    # Check if the folder exists
    if not os.path.exists(folder_path):
        raise FileNotFoundError(f"The specified folder does not exist: {folder_path}")

    # List comprehension to gather all image paths with the specified extensions
    image_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(extensions)]

    return image_paths

In [ ]:
# Function to convert to gray scale
def load_and_preprocess_images(image_paths, resize_dim=(256, 256)):
    images = []
    image_ids = []

    # Initialize tqdm progress bar
    for path in tqdm(image_paths, desc="Loading and preprocessing images", unit="image"):
        img = cv2.imread(path)
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
        # img_resized = cv2.resize(img_gray, resize_dim)  # Resize for consistency
        images.append(img_gray)
        image_ids.append(f'Image_{len(images)}')  # Assign image ID as Image_1, Image_2, etc.

    return images, image_ids

In [ ]:
# Load the CSV file with the image statistics
images_stats_path = "../data-understanding/images_stats.csv"  
images_stats_df = pd.read_csv(images_stats_path)

In [ ]:
images_stats_df.columns

## Functions per step

### Step 1: Noise Reduction Techniques

In [ ]:
# Noise Reduction Functions
def apply_gaussian_blur(image, ksize=(5, 5)):
    """Apply Gaussian Blur to reduce noise with the specified kernel size."""
    return cv2.GaussianBlur(image, ksize, 0)

def apply_median_blur(image, ksize=5):
    """Apply Median Blur to reduce salt-and-pepper noise with the specified kernel size."""
    return cv2.medianBlur(image, ksize)

def apply_non_local_means(image, h=10, templateWindowSize=7, searchWindowSize=21):
    """Apply Non-Local Means Denoising with specified parameters."""
    return cv2.fastNlMeansDenoising(image, None, h, templateWindowSize, searchWindowSize)

### Step 2: Histogram Equalization Techniques

In [ ]:
# Histogram Equalization Functions
def apply_histogram_equalization(image):
    return cv2.equalizeHist(image)

def apply_clahe(image, clipLimit=2.0, tileGridSize=(8, 8)):
    """Apply CLAHE to enhance image contrast with specified parameters."""
    clahe = cv2.createCLAHE(clipLimit=clipLimit, tileGridSize=tileGridSize)
    return clahe.apply(image)

### Step 3: Binarization Techniques

In [ ]:
# Binarization Functions
def apply_global_threshold(image, thresholdValue=127):
    """Apply Global Thresholding with the specified threshold value."""
    _, binary_image = cv2.threshold(image, thresholdValue, 255, cv2.THRESH_BINARY)
    return binary_image

def apply_adaptive_threshold(image, adaptiveMethod=cv2.ADAPTIVE_THRESH_MEAN_C, blockSize=11, C=2):
    """Apply Adaptive Thresholding with the specified method, block size, and constant C."""
    return cv2.adaptiveThreshold(image, 255, adaptiveMethod, cv2.THRESH_BINARY, blockSize, C)

def apply_otsu_threshold(image):
    """Apply Otsu Thresholding."""
    _, binary_image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return binary_image

def apply_inverted_otsu_threshold(image):
    """Apply Inverted Otsu Thresholding."""
    _, binary_image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    return binary_image


### Step 4: Morphological Operations Techniques

In [ ]:
# Morphological Operations Functions
def apply_dilation(image, kernel_size=(5, 5)):
    """Apply Dilation with the specified kernel size."""
    kernel = np.ones(kernel_size, np.uint8)
    return cv2.dilate(image, kernel, iterations=1)

def apply_erosion(image, kernel_size=(5, 5)):
    """Apply Erosion with the specified kernel size."""
    kernel = np.ones(kernel_size, np.uint8)
    return cv2.erode(image, kernel, iterations=1)

def apply_opening(image, kernel_size=(5, 5)):
    """Apply Morphological Opening with the specified kernel size."""
    kernel = np.ones(kernel_size, np.uint8)
    return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

def apply_closing(image, kernel_size=(5, 5)):
    """Apply Morphological Closing with the specified kernel size."""
    kernel = np.ones(kernel_size, np.uint8)
    return cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)


### Step 5: Edge Detection Techniques

In [ ]:
# Edge Detection Functions
def apply_canny_edge(image, threshold1=50, threshold2=150):
    """Apply Canny Edge Detection with specified thresholds."""
    return cv2.Canny(image, threshold1, threshold2)

def apply_sobel_edge(image, ksize=3, scale=1, delta=0, borderType=cv2.BORDER_DEFAULT):
    """Apply Sobel Edge Detection with specified parameters."""
    return cv2.Sobel(image, cv2.CV_64F, 1, 1, ksize=ksize, scale=scale, delta=delta, borderType=borderType)

def apply_unsharp_masking(image, amount=1.5, kernel_size=(0, 0)):
    """Apply Unsharp Masking to sharpen the image."""
    blurred = cv2.GaussianBlur(image, kernel_size, 0)
    sharpened = cv2.addWeighted(image, 1 + amount, blurred, -amount, 0)
    return sharpened

## Characteristics Calculation for testing

In [ ]:
# Image Characteristics Calculation Functions - from data understanding it2
def calculate_brightness(image):
    return np.mean(image)

def calculate_sharpness(image):
    return cv2.Laplacian(image, cv2.CV_64F).var()

def calculate_contrast(image):
    return image.std()

def calculate_noise(image):
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(image, (3, 3), 0)
    noise = cv2.absdiff(image, blurred)
    return np.var(noise)

# def calculate_skew(image):
#     if len(image.shape) != 2:
#         raise ValueError("Invalid image format. Image must be a 2D grayscale image.")
#     _, binary = cv2.threshold(image, 150, 255, cv2.THRESH_BINARY_INV)
#     coords = np.column_stack(np.where(binary > 0))
#     if coords.size == 0:
#         return 0
#     angle = cv2.minAreaRect(coords)[-1]
#     if angle < -45:
#         angle = -(90 + angle)
#     else:
#         angle = -angle
#     if abs(angle) < 1e-2:
#         angle = 0
#     return round(angle, 2)
# 
# def calculate_line_spacing(image):
#     if len(image.shape) != 2:
#         raise ValueError("Invalid image format. Image must be a 2D grayscale image.")
#     _, binary = cv2.threshold(image, 150, 255, cv2.THRESH_BINARY_INV)
#     contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     heights = [cv2.boundingRect(contour)[3] for contour in contours]
#     if len(heights) > 1:
#         line_spacing = np.mean(np.diff(sorted(heights)))
#     else:
#         line_spacing = 0
#     return line_spacing
# 
# def detect_tables(image):
#     if len(image.shape) != 2:
#         raise ValueError("Invalid image format. Image must be a 2D grayscale image.")
#     _, binary = cv2.threshold(image, 150, 255, cv2.THRESH_BINARY_INV)
#     binary = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
#     contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     table_contours = [contour for contour in contours if cv2.contourArea(contour) > 1000]
#     return len(table_contours)
# 
# def calculate_resolution(image):
#     height, width = image.shape[:2]
#     return height * width
# 
# def calculate_elements_detection(image):
#     if len(image.shape) != 2:
#         raise ValueError("Invalid image format. Image must be a 2D grayscale image.")
#     _, binary = cv2.threshold(image, 150, 255, cv2.THRESH_BINARY_INV)
#     contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     return len(contours)
# 
# def calculate_texture(image):
#     laplacian = cv2.Laplacian(image, cv2.CV_64F)
#     return laplacian.std()
# 
# def calculate_patterns(image):
#     if len(image.shape) != 2:
#         raise ValueError("Invalid image format. Image must be a 2D grayscale image.")
#     edges = cv2.Canny(image, 100, 200)
#     return np.sum(edges > 0)


## Evaluation per step

### Function of evaluation

In [ ]:
# Basic Evaluation Function
# -------------------------
# def basic_evaluation(image, techniques_dict, original_stats):
#     evaluation_results = {}
#     for technique_name, technique_func in techniques_dict.items():
#         processed_image = technique_func(image)
#         stats = {
#             "Brightness": calculate_brightness(processed_image),
#             "Sharpness": calculate_sharpness(processed_image),
#             "Contrast": calculate_contrast(processed_image),
#             "Noise": calculate_noise(processed_image)
#         }
# 
#         # Basic scoring function - prioritizing sharpness, contrast, and minimized noise
#         score = stats["Sharpness"] + stats["Contrast"] - stats["Noise"]
#         evaluation_results[technique_name] = {"Score": score, "Stats": stats}
# 
#     best_technique = max(evaluation_results, key=lambda x: evaluation_results[x]["Score"])
#     return {"Best Technique": best_technique, "Evaluation Results": evaluation_results}


In [ ]:
# def advanced_evaluation(image, techniques_dict, original_stats):
#     evaluation_results = {}
# 
#     for technique_name, technique_func in techniques_dict.items():
#         # Apply the technique
#         processed_image = technique_func(image)
# 
#         # Calculate characteristics for the processed image
#         stats = {
#             "Brightness": calculate_brightness(processed_image),
#             "Sharpness": calculate_sharpness(processed_image),
#             "Contrast": calculate_contrast(processed_image),
#             "Noise": calculate_noise(processed_image),
#             "Skew": calculate_skew(processed_image),
#             "Line Spacing": calculate_line_spacing(processed_image),
#             "Tables Detected": detect_tables(processed_image),
#             "Resolution": calculate_resolution(processed_image),
#             "Detected Elements": calculate_elements_detection(processed_image),
#             "Texture": calculate_texture(processed_image),
#             "Patterns": calculate_patterns(processed_image)
#         }
# 
#         # Normalize metrics to comparable ranges (between 0 and 1, roughly)
#         stats_normalized = {
#             "Brightness": stats["Brightness"] / 255,
#             "Sharpness": stats["Sharpness"] / 1000,
#             "Contrast": stats["Contrast"] / 255,
#             "Noise": stats["Noise"] / 255,
#             "Skew": stats["Skew"] / 45,
#             "Line Spacing": stats["Line Spacing"] / 100,
#             "Tables Detected": stats["Tables Detected"] / 10,
#             "Resolution": stats["Resolution"] / (512 * 512),
#             "Detected Elements": stats["Detected Elements"] / 100,
#             "Texture": stats["Texture"] / 100,
#             "Patterns": stats["Patterns"] / 1000
#         }
# 
#         # Normalize the original stats for comparison
#         original_stats_normalized = {
#             "Brightness": original_stats["Brightness"] / 255,
#             "Sharpness": original_stats["Sharpness"] / 1000,
#             "Contrast": original_stats["Contrast"] / 255,
#             "Noise": original_stats["Noise"] / 255,
#             "Skew": original_stats["Skew"] / 45,
#             "Line Spacing": original_stats["Line Spacing"] / 100,
#             "Tables Detected": original_stats["Tables Detected"] / 10,
#             "Resolution": original_stats["Resolution"] / (512 * 512),
#             "Detected Elements": original_stats["Detected Elements"] / 100,
#             "Texture": original_stats["Texture"] / 100,
#             "Patterns": original_stats["Patterns"] / 1000
#         }
# 
#         # Weights for each characteristic (to determine their importance)
#         weights = {
#             "Brightness": -1.0,  # Closer to original is better (penalized if different)
#             "Sharpness": 2.0,    # Higher is better (rewarded if improved)
#             "Contrast": 1.0,     # Higher is better (rewarded if improved)
#             "Noise": -1.5,       # Lower is better (penalized if increased)
#             "Skew": -0.5,        # Closer to original is better (penalized if different)
#             "Line Spacing": -0.5,  # Closer to original is better (penalized if different)
#             "Tables Detected": 1.0,  # More tables detected is better
#             "Resolution": 1.0,    # Higher is better
#             "Detected Elements": 1.0,  # More elements detected is better
#             "Texture": 1.0,       # Higher texture complexity is better
#             "Patterns": 1.0       # More patterns detected is better
#         }
# 
#         # Calculate score using normalized metrics and weights
#         score = 0
#         for metric, value in stats_normalized.items():
#             original_value = original_stats_normalized.get(metric, 0)
#             score += weights[metric] * (value - original_value)
# 
#         evaluation_results[technique_name] = {"Score": score, "Stats": stats}
# 
#     # Determine the best technique based on the highest score
#     best_technique = max(evaluation_results, key=lambda x: evaluation_results[x]["Score"])
#     return {"Best Technique": best_technique, "Evaluation Results": evaluation_results}


##### Normalization Process

**Normalization** is crucial for ensuring that the values of different characteristics (`Brightness`, `Sharpness`, `Contrast`, `Noise`) are on a similar scale. Without normalization, these characteristics might have vastly different ranges, which could skew the evaluation. Here's what happens in the function:

1. **Brightness Normalization**:
   - The brightness of an image is typically represented on a scale from 0 to 255 (as an 8-bit grayscale value).
   - To normalize brightness, we divide it by 255, which brings its range between 0 and 1.

2. **Sharpness Normalization**:
   - Sharpness is measured as the variance of the Laplacian, which often has larger values than brightness.
   - Dividing by `1000` helps normalize it to roughly between 0 and 1. The choice of `1000` is made to ensure that sharpness values are comparable to the other metrics.

3. **Contrast Normalization**:
   - Contrast is calculated using the standard deviation of pixel values, which usually falls between 0 and 255 for 8-bit images.
   - Dividing by 255 brings contrast into the range between 0 and 1.

4. **Noise Normalization**:
   - The noise measure is the variance of the difference between the original and a blurred version of the image.
   - Dividing by 255 brings it to a similar range as the other characteristics, ensuring comparability.

By normalizing all metrics to a range between 0 and 1, we ensure that each characteristic has equal weight in the evaluation, preventing one metric from dominating due to a larger numeric range.

##### Scoring Calculation

Once all metrics are normalized, a score is calculated to determine how well the processed image improves compared to the original. Here's the breakdown of the scoring process:

1. **Weights for Characteristics**:
   - We assign **weights** to each metric based on its importance:
     - **Brightness**: Weight of `-1.0` means that deviation from the original value is penalized.
     - **Sharpness**: Weight of `2.0` rewards increased sharpness.
     - **Contrast**: Weight of `1.0` rewards increased contrast.
     - **Noise**: Weight of `-1.5` penalizes increased noise.

2. **Score Calculation**:
   - The difference between the normalized processed value and the normalized original value is multiplied by the respective weight.
   - If a **positively weighted metric** (like sharpness or contrast) **improves**, it contributes positively to the score.
   - If a **negatively weighted metric** (like noise or brightness deviation) **increases**, it contributes negatively, penalizing the score.

3. **Best Technique Selection**:
   - After calculating the score for each technique, the function selects the one with the **highest score**.


In [ ]:
def advanced_evaluation(image, techniques_dict, original_stats):
    evaluation_results = {}

    for technique_name, technique_func in techniques_dict.items():
        # Apply the technique
        processed_image = technique_func(image)

        # Calculate characteristics for the processed image
        stats = {
            "Brightness": calculate_brightness(processed_image),
            "Sharpness": calculate_sharpness(processed_image),
            "Contrast": calculate_contrast(processed_image),
            "Noise": calculate_noise(processed_image),
        }

        # Normalize metrics to comparable ranges (between 0 and 1, roughly)
        stats_normalized = {
            "Brightness": stats["Brightness"] / 255,
            "Sharpness": stats["Sharpness"] / 1000,
            "Contrast": stats["Contrast"] / 255,
            "Noise": stats["Noise"] / 255,
        }

        # Normalize the original stats for comparison
        original_stats_normalized = {
            "Brightness": original_stats["Brightness"] / 255,
            "Sharpness": original_stats["Sharpness"] / 1000,
            "Contrast": original_stats["Contrast"] / 255,
            "Noise": original_stats["Noise"] / 255,
        }

        # Weights for each characteristic (to determine their importance)
        weights = {
            "Brightness": 1.0,  # Higher is better (rewarded if improved)
            "Sharpness": 1.0,    # Higher is better (rewarded if improved) but images were generally sharp already 
            "Contrast": 2.0,     # Higher is better (rewarded if improved) the levels of contrast were lower and obstructed details
            "Noise": -1.5,       # Lower is better (penalized if increased)
        }

        # Calculate score using normalized metrics and weights
        score = 0
        for metric, value in stats_normalized.items():
            original_value = original_stats_normalized.get(metric, 0)
            score += weights[metric] * (value - original_value)

        evaluation_results[technique_name] = {"Score": score, "Stats": stats}

    # Determine the best technique based on the highest score
    best_technique = max(evaluation_results, key=lambda x: evaluation_results[x]["Score"])
    return {"Best Technique": best_technique, "Evaluation Results": evaluation_results}


In [ ]:
# Function for Each Step testing
def run_step(step_name, techniques_dict, test_images, test_image_ids, best_techniques_list):
    print(f"\nRunning Step: {step_name}\n{'-' * 40}")
    all_results = []

    # Folder to save images from this step
    output_folder_step = f"./Data/It2/{step_name}"
    os.makedirs(output_folder_step, exist_ok=True)

    # Image ID to be saved for comparison (using the first image ID consistently for all techniques)
    save_image_id = test_image_ids[0] if len(test_image_ids) > 0 else None

    for img, img_id in zip(test_images, test_image_ids):
        # Retrieve original stats from the dataset for the specific image being processed
        original_stats = images_stats_df[images_stats_df['Image'] == img_id].iloc[0].to_dict()

        # Evaluate each technique on the current image
        step_result = advanced_evaluation(img, techniques_dict, original_stats)
        all_results.append((img_id, original_stats, step_result))
        print(f"Best Technique for {img_id}: {step_result['Best Technique']}")

        # Save one processed image per technique per step (consistent image ID across all techniques)
        if img_id == save_image_id:
            for technique_name, technique_func in techniques_dict.items():
                # Apply the technique to the image
                processed_image = technique_func(img)

                # Save the processed image
                image_save_path = f"{output_folder_step}/{technique_name}_Image_{img_id}.jpg"
                cv2.imwrite(image_save_path, processed_image)

    # Generate Comparison Table
    comparison_data = []
    for img_id, original_stats, result in all_results:
        # Add original stats row
        comparison_data.append([img_id, "Original"] + list(original_stats.values())[1:])  # Skip the 'Image' key
        # Add each technique's stats
        for technique, metrics in result["Evaluation Results"].items():
            comparison_data.append([img_id, f"{step_name} - {technique}"] + list(metrics["Stats"].values()))

    # Create a DataFrame to store the results
    comparison_df = pd.DataFrame(comparison_data, columns=[
        "Image_ID", "Technique", "Brightness", "Sharpness", "Contrast", "Noise"
    ])

    # Generate Recommendation
    recommended_technique_name = max(all_results, key=lambda x: x[2]["Evaluation Results"][x[2]["Best Technique"]]["Score"])[2]["Best Technique"]
    recommended_technique_func = techniques_dict[recommended_technique_name]
    print(f"\nRecommended Technique for {step_name}: {recommended_technique_name}\n")

    # Append both technique name and function for further tuning
    best_techniques_list.append((step_name, recommended_technique_name, recommended_technique_func))

    # Return the comparison DataFrame
    return comparison_df

### Running Different Techniques per Step

In [ ]:
# Load all image file paths from the specified folder
image_paths_all = load_images_from_folder(folder_path)

# Load and preprocess all images
total_images, total_image_ids = load_and_preprocess_images(image_paths_all)

# Randomly select 5 images for experimentation
experiment_indices = random.sample(range(len(total_images)), 5)
test_images = [total_images[i] for i in experiment_indices]
test_image_ids = [total_image_ids[i] for i in experiment_indices]


In [ ]:
best_techniques_list = []
comparison_tables = []

In [ ]:
# Step 1: Noise Reduction
noise_reduction_techniques = {
    "Gaussian Blur": lambda img: cv2.GaussianBlur(img, (5, 5), 0),
    "Median Blur": lambda img: cv2.medianBlur(img, 5),
    "Non-Local Means": lambda img: cv2.fastNlMeansDenoising(img, None, 10, 7, 21)
}
comparison_tables.append(run_step("Noise Reduction", noise_reduction_techniques, test_images, test_image_ids, best_techniques_list))

In [ ]:
# Step 2: Histogram Equalization
histogram_equalization_techniques = {
    "Histogram Equalization": lambda img: cv2.equalizeHist(img),
    "CLAHE": lambda img: cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8)).apply(img)
}
comparison_tables.append(run_step("Histogram Equalization", histogram_equalization_techniques, test_images, test_image_ids, best_techniques_list))

In [ ]:
# Step 3: Binarization
binarization_techniques = {
    "Global Threshold": lambda img: cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)[1],
    "Adaptive Threshold": lambda img: cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2),
    "Otsu Threshold": lambda img: cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1],
    "Inverted Otsu Threshold": lambda img: cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
}
comparison_tables.append(run_step("Binarization", binarization_techniques, test_images, test_image_ids, best_techniques_list))


In [ ]:
# Step 4: Morphological Operations
morphological_operations_techniques = {
    "Dilation": lambda img: cv2.dilate(img, np.ones((5, 5), np.uint8), iterations=1),
    "Erosion": lambda img: cv2.erode(img, np.ones((5, 5), np.uint8), iterations=1),
    "Opening": lambda img: cv2.morphologyEx(img, cv2.MORPH_OPEN, np.ones((5, 5), np.uint8)),
    "Closing": lambda img: cv2.morphologyEx(img, cv2.MORPH_CLOSE, np.ones((5, 5), np.uint8))
}
comparison_tables.append(run_step("Morphological Operations", morphological_operations_techniques, test_images, test_image_ids, best_techniques_list))

In [ ]:
# Step 5: Edge Detection
edge_detection_techniques = {
    "Canny Edge": lambda img: cv2.Canny(img, 100, 200),
    "Sobel Edge": lambda img: cv2.convertScaleAbs(cv2.Sobel(img, cv2.CV_64F, 1, 1, ksize=3)),
    "Unsharp Mask": lambda img: cv2.addWeighted(img, 1.5, cv2.GaussianBlur(img, (0, 0), 3), -0.5, 0)
}
comparison_tables.append(run_step("Edge Detection", edge_detection_techniques, test_images, test_image_ids, best_techniques_list))


## Final best techniques per step

In [ ]:
# Print the list of best techniques for each step
print("\nBest Techniques for Each Step:")
for step, technique_name, technique_func in best_techniques_list:
    print(f"{step}: {technique_name}")

In [ ]:
for i, comparison_df in enumerate(comparison_tables):
    comparison_df.to_csv(f"comparison tables/comparison_table_step_{i+1}.csv", index=False)

In [ ]:
# Display comparison tables within the notebook
for i, comparison_df in enumerate(comparison_tables):
    print(f"Comparison Table for Step {i+1}:")
    display(comparison_df.head())

In [ ]:
# Generate Average Comparison Table
average_comparison_data = []
for comparison_df in comparison_tables:
    avg_stats = comparison_df.groupby("Technique").mean().reset_index()
    average_comparison_data.append(avg_stats)

# Combine average stats from all steps
average_comparison_df = pd.concat(average_comparison_data, ignore_index=True)
# Save the average comparison table to a CSV file
average_comparison_df.to_csv("comparison tables/average_comparison_table.csv", index=False)

In [ ]:
average_comparison_df

## Hyperparameter Tuning

In [ ]:
# Hyperparameter Tuning Function
def hyperparameter_tuning(images, best_techniques_list, param_grids, evaluation_function):
    tuned_results = {}

    # Choose one image to save for comparison purposes
    save_image_id = total_image_ids[0] if len(total_image_ids) > 0 else None

    for step_name, technique_name, best_technique_func in best_techniques_list:
        print(f"\nHyperparameter Tuning for Step: {step_name}\n{'-' * 40}")
        best_params = None
        best_score = -np.inf
        param_grid = param_grids.get(technique_name, [])

        for params in param_grid:
            total_score = 0

            # Convert parameter dict to string for filename (e.g., 'param1_val1_param2_val2')
            params_str = "_".join([f"{key}_{value}" for key, value in params.items()])

            for img, img_id in zip(images, total_image_ids):
                try:
                    # Apply the best technique with the given parameters explicitly based on technique name
                    if technique_name == "Gaussian Blur":
                        processed_image = apply_gaussian_blur(img, **params)
                    elif technique_name == "Median Blur":
                        processed_image = apply_median_blur(img, **params)
                    elif technique_name == "Non-Local Means":
                        processed_image = apply_non_local_means(img, **params)
                    elif technique_name == "CLAHE":
                        processed_image = apply_clahe(img, **params)
                    elif technique_name == "Global Threshold":
                        processed_image = apply_global_threshold(img, **params)
                    elif technique_name == "Adaptive Threshold":
                        processed_image = apply_adaptive_threshold(img, **params)
                    elif technique_name == "Otsu Threshold":
                        processed_image = apply_otsu_threshold(img)
                    elif technique_name == "Inverted Otsu Threshold":
                        processed_image = apply_inverted_otsu_threshold(img)
                    elif technique_name == "Dilation":
                        processed_image = apply_dilation(img, **params)
                    elif technique_name == "Erosion":
                        processed_image = apply_erosion(img, **params)
                    elif technique_name == "Morphological Opening":
                        processed_image = apply_opening(img, **params)
                    elif technique_name == "Morphological Closing":
                        processed_image = apply_closing(img, **params)
                    elif technique_name == "Canny Edge":
                        processed_image = apply_canny_edge(img, **params)
                    elif technique_name == "Sobel Edge":
                        processed_image = apply_sobel_edge(img, **params)
                    elif technique_name == "Unsharp Masking":
                        processed_image = apply_unsharp_masking(img, **params)
                    else:
                        raise ValueError(f"Unknown technique: {technique_name}")

                except TypeError as e:
                    print(f"Skipping parameters {params} due to TypeError: {e}")
                    continue

                # Retrieve original stats for comparison
                original_stats = images_stats_df[images_stats_df['Image'] == img_id].iloc[0].to_dict()
                evaluation_result = evaluation_function(processed_image, {technique_name: best_technique_func}, original_stats)
                step_score = evaluation_result["Evaluation Results"][technique_name]["Score"]
                total_score += step_score

                # Save the processed image if it is the one designated for saving
                if img_id == save_image_id:
                    # Specify folder for best techniques and their parameters
                    output_folder_tuning = f"./Data/It2/Best Techniques/{step_name}_{technique_name}"
                    os.makedirs(output_folder_tuning, exist_ok=True)

                    # Save the processed image
                    cv2.imwrite(f"{output_folder_tuning}/{technique_name}_Params_{params_str}_Image_{img_id}.jpg", processed_image)

            avg_score = total_score / len(images) if len(images) > 0 else -np.inf

            if avg_score > best_score:
                best_score = avg_score
                best_params = params

            print(f"Parameters: {params}, Score: {avg_score}")

        tuned_results[step_name] = {
            "Best Parameters": best_params,
            "Best Score": best_score
        }
        print(f"Best Parameters for {step_name}: {best_params} with Score: {best_score}\n")

    return tuned_results


#### Explanation of Techniques and Parameters

##### 1. Noise Reduction
###### Gaussian Blur (`ksize`)
- **Parameter**: `ksize` (kernel size)
- **Meaning**: Defines the extent of smoothing applied. A small kernel (e.g., `(3, 3)`) produces minimal blurring, preserving details, while larger kernels (e.g., `(9, 9)`) apply more significant blurring, which is useful for reducing noise but may remove finer details.
- **Range**:
  - `(3, 3)`, `(5, 5)`, `(7, 7)`, `(9, 9)`
  - Smaller sizes preserve more details, larger sizes reduce noise more aggressively.

###### Non-Local Means (`h`, `templateWindowSize`, `searchWindowSize`)
- **Parameters**:
  - `h`: Filtering strength (higher values = stronger filtering).
  - `templateWindowSize`: Size of the patch used for comparison.
  - `searchWindowSize`: Size of the window around the pixel for searching similar patches.
- **Range**:
  - `h`: `5`, `10`, `15`, `20`
  - `templateWindowSize`: `7`, `10`
  - `searchWindowSize`: `21`, `31`
  - Balances noise reduction quality and processing time.

###### Median Blur (`ksize`)
- **Parameter**: `ksize` (kernel size)
- **Meaning**: Reduces "salt-and-pepper" noise by replacing each pixel with the median of neighboring pixels. Larger kernels apply stronger noise reduction, potentially losing details.
- **Range**:
  - `3`, `5`, `7`, `9`
  - Smaller values (`3`, `5`) are useful for mild noise; larger values (`7`, `9`) are effective for more significant noise.

##### 2. Histogram Equalization
###### CLAHE (`clipLimit`, `tileGridSize`)
- **Parameters**:
  - `clipLimit`: Controls contrast enhancement limit.
  - `tileGridSize`: Size of the grid for local histogram equalization.
- **Range**:
  - `clipLimit`: `2.0` to `6.0`
  - `tileGridSize`: `(4, 4)`, `(6, 6)`, `(8, 8)`
  - Lower `clipLimit` values reduce noise amplification, larger `tileGridSize` produces smoother results.

##### 3. Binarization
###### Global Threshold (`thresholdValue`)
- **Parameter**: `thresholdValue`
- **Meaning**: Used to convert grayscale images to binary by comparing pixel values to a threshold. Lower values produce more white areas.
- **Range**:
  - `100`, `127`, `150`, `200`
  - Balances the separation between foreground and background.

###### Adaptive Threshold (`adaptiveMethod`, `blockSize`, `C`)
- **Parameters**:
  - `adaptiveMethod`: The method used for calculating the threshold (`cv2.ADAPTIVE_THRESH_MEAN_C` or `cv2.ADAPTIVE_THRESH_GAUSSIAN_C`).
  - `blockSize`: Size of the local area considered for thresholding.
  - `C`: Constant subtracted from the mean or weighted sum.
- **Range**:
  - `adaptiveMethod`: `cv2.ADAPTIVE_THRESH_MEAN_C` or `cv2.ADAPTIVE_THRESH_GAUSSIAN_C`
  - `blockSize`: `11`, `15`
  - `C`: `2`, `3`
  - Allows adjustment to local image variations for better segmentation.

###### Otsu Threshold
- **Meaning**: Automatically determines the optimal threshold value to convert grayscale images to binary.
- **Use**: Effective for images with bimodal histograms, making it suitable for foreground and background separation.
- **Parameters**: None, Otsu's method calculates the optimal threshold automatically.

###### Inverted Otsu Threshold
- **Meaning**: Applies Otsu's method for thresholding but inverts the resulting binary image, making foreground black and background white.
- **Use**: Useful when the target regions are originally white on a dark background.
- **Parameters**: None, as Otsu's method calculates the optimal threshold automatically.

##### 4. Morphological Operations
###### Operation (`MORPH_OPEN`, `MORPH_CLOSE`, `DILATE`, `ERODE`, `kernel_size`)
- **Parameters**:
  - `operation`: The morphological transformation to apply.
    - `cv2.MORPH_OPEN`: Removes small white noise.
    - `cv2.MORPH_CLOSE`: Fills small black gaps in white areas.
    - `cv2.MORPH_DILATE`: Expands white areas to connect small features.
    - `cv2.MORPH_ERODE`: Shrinks white areas to reduce noise.
  - `kernel_size`: Size of the structuring element.
- **Range**:
  - `kernel_size`: `(3, 3)`, `(5, 5)`, `(7, 7)`, `(9, 9)`
  - Larger kernels apply more aggressive changes for connecting, removing, or shrinking features.

##### 5. Edge Detection
###### Canny Edge Detection (`threshold1`, `threshold2`)
- **Parameters**:
  - `threshold1`: Lower threshold for weak edges.
  - `threshold2`: Upper threshold for strong edges.
- **Range**:
  - `threshold1` and `threshold2`: `(50, 150)`, `(100, 200)`, `(150, 250)`, `(200, 300)`
  - Lower values detect more edges, useful for detailed images; higher values highlight stronger, more defined edges.

###### Sobel Edge Detection (`ksize`, `scale`, `delta`, `borderType`)
- **Parameters**:
  - `ksize`: Kernel size for the Sobel operator.
  - `scale`: Scaling factor for gradients.
  - `delta`: Value added to the result.
  - `borderType`: Border handling for edges.
- **Range**:
  - `ksize`: `3`, `5`, `7`
  - `scale`: `1`, `2`
  - `delta`: `0` (default)
  - `borderType`: `cv2.BORDER_DEFAULT`
  - Adjusts the level of detail and sharpness captured by the filter.

###### Unsharp Masking (`amount`, `kernel_size`)
- **Parameters**:
  - `amount`: Strength of sharpening effect applied to the image.
  - `kernel_size`: Size of the kernel used for blurring in the unsharp mask process.
- **Range**:
  - `amount`: `1.0` to `2.5` (Higher values produce stronger sharpening)
  - `kernel_size`: `(3, 3)`, `(5, 5)`, `(7, 7)`, `(9, 9)`
  - Sharpening enhances edges and contrasts to make features more prominent, but excessively high values can introduce artifacts.


In [ ]:
# Define expanded parameter grids for hyperparameter tuning for each technique
technique_param_grids = {
    # Noise Reduction Techniques Parameters
    # Gaussian Blur - kernel size affects the degree of blurring
    "Gaussian Blur": [
        {"ksize": (3, 3)},  # Small blur, preserves more details while reducing minor noise
        {"ksize": (5, 5)},  # Moderate blur, balances noise reduction and detail preservation
        {"ksize": (7, 7)},  # Stronger blur, reduces more noise but may lose more details
        {"ksize": (9, 9)}   # High blur, significant reduction of noise, more detail loss
    ],

    # Median Blur - kernel size affects the reduction of salt-and-pepper noise
    "Median Blur": [
        {"ksize": 3},  # Small kernel, effective for minor salt-and-pepper noise
        {"ksize": 5},  # Moderate kernel, more aggressive noise reduction
        {"ksize": 7},  # Large kernel, used for significant salt-and-pepper noise reduction
        {"ksize": 9}   # Largest kernel, aggressive noise reduction but may lose finer details
    ],

    # Non-Local Means - affects noise reduction strength and quality
    "Non-Local Means": [
        {"h": 5, "templateWindowSize": 7, "searchWindowSize": 21},   # Low filter strength (h), smaller template
        {"h": 10, "templateWindowSize": 7, "searchWindowSize": 21},  # Moderate filter strength (h), balance of denoising and details
        {"h": 15, "templateWindowSize": 7, "searchWindowSize": 21},  # Strong filter strength, more noise reduction but risk of over-smoothing
        {"h": 20, "templateWindowSize": 10, "searchWindowSize": 31}  # Higher strength and larger search windows for stronger denoising
    ],

    # Histogram Equalization Techniques Parameters
    # CLAHE - clip limit controls contrast, tile grid size controls local regions
    "CLAHE": [
        {"clipLimit": 2.0, "tileGridSize": (8, 8)},  # Low clip limit, preserves global contrast, effective for mild contrast enhancement
        {"clipLimit": 3.0, "tileGridSize": (8, 8)},  # Moderate clip limit, better enhancement for darker/lighter regions
        {"clipLimit": 4.0, "tileGridSize": (4, 4)},  # Higher clip limit, can lead to artifacts but increases local contrast
        {"clipLimit": 5.0, "tileGridSize": (6, 6)}   # High clip limit, strong local contrast enhancement
    ],

    # Binarization Techniques Parameters
    # Global Threshold - value for the threshold, used to separate foreground from background
    "Global Threshold": [
        {"thresholdValue": 100},  # Low threshold, makes more areas white, may overexpose
        {"thresholdValue": 127},  # Middle threshold, balance between foreground and background
        {"thresholdValue": 150},  # High threshold, less white, more black areas
        {"thresholdValue": 200}   # Higher threshold, darkest parts retained as foreground
    ],

    # Adaptive Threshold - block size and constant C, used for adaptive thresholding
    "Adaptive Threshold": [
        {"adaptiveMethod": cv2.ADAPTIVE_THRESH_MEAN_C, "blockSize": 11, "C": 2},  # Small block size, captures smaller variations
        {"adaptiveMethod": cv2.ADAPTIVE_THRESH_MEAN_C, "blockSize": 15, "C": 3},  # Larger block size, averages larger areas
        {"adaptiveMethod": cv2.ADAPTIVE_THRESH_GAUSSIAN_C, "blockSize": 11, "C": 2},  # Gaussian weighting, better for uneven lighting
        {"adaptiveMethod": cv2.ADAPTIVE_THRESH_GAUSSIAN_C, "blockSize": 15, "C": 3}   # Larger area, smoother output
    ],

    "Otsu Threshold": [
        {}  # No parameters needed, automatic threshold selection
    ],
    
    "Inverted Otsu Threshold": [{}],

    # Morphological Operations Techniques Parameters
    # Dilation - kernel size affects how much an object is expanded, helps to highlight and connect features in the image
    "Dilation": [
        {"kernel_size": (3, 3)},  # Small kernel, slight expansion of features
        {"kernel_size": (5, 5)},  # Medium kernel, moderate expansion, often used to fill small holes
        {"kernel_size": (7, 7)},  # Larger kernel, more significant expansion, fills larger gaps
        {"kernel_size": (9, 9)}   # Largest kernel, aggressive expansion, can connect disjoint parts
    ],

    # Erosion - kernel size affects how much an object is eroded, used to reduce noise by shrinking foreground areas
    "Erosion": [
        {"kernel_size": (3, 3)},  # Small kernel, minimal shrinking of features
        {"kernel_size": (5, 5)},  # Medium kernel, reduces small noise while keeping the main features intact
        {"kernel_size": (7, 7)},  # Larger kernel, removes more fine details, useful for stronger noise reduction
        {"kernel_size": (9, 9)}   # Largest kernel, aggressive erosion, may result in significant information loss
    ],

    # Morphological Opening - kernel size affects noise removal, used for removing small white noise from black backgrounds
    "Morphological Opening": [
        {"kernel_size": (3, 3)},  # Small kernel, removes small white noise but keeps the main structure
        {"kernel_size": (5, 5)},  # Medium kernel, better noise removal, may affect finer details
        {"kernel_size": (7, 7)}   # Larger kernel, stronger noise reduction, potentially removes small features
    ],

    # Morphological Closing - kernel size affects how gaps in foreground objects are filled, used to close small black holes within objects
    "Morphological Closing": [
        {"kernel_size": (3, 3)},  # Small kernel, fills tiny holes, maintains object shape
        {"kernel_size": (5, 5)},  # Medium kernel, closes medium-sized gaps, useful for refining object borders
        {"kernel_size": (7, 7)}   # Larger kernel, aggressively closes gaps, useful for solidifying larger structures
    ],

    # Edge Detection Techniques Parameters
    # Canny Edge Detection - lower and upper thresholds for edge linking
    "Canny Edge": [
        {"threshold1": 50, "threshold2": 150},  # Low thresholds, more edges detected
        {"threshold1": 100, "threshold2": 200},  # Moderate thresholds, balanced edge detection
        {"threshold1": 150, "threshold2": 250},  # High thresholds, only strong edges detected
        {"threshold1": 200, "threshold2": 300}   # Very high thresholds, detects fewer edges, focused on major features
    ],

    # Sobel Edge - kernel size, scale, delta, border type for Sobel edge detection
    "Sobel Edge": [
        {"ksize": 3, "scale": 1, "delta": 0, "borderType": cv2.BORDER_DEFAULT},  # Small kernel, detects finer details
        {"ksize": 5, "scale": 1, "delta": 0, "borderType": cv2.BORDER_DEFAULT},  # Medium kernel, balances detail and noise suppression
        {"ksize": 7, "scale": 1, "delta": 0, "borderType": cv2.BORDER_DEFAULT},  # Larger kernel, captures broader gradients
        {"ksize": 3, "scale": 2, "delta": 0, "borderType": cv2.BORDER_DEFAULT}   # Increased scale, emphasizes detected gradients more strongly
    ],
    "Unsharp Masking": [
        {"amount": 1.0, "kernel_size": (3, 3)},
        {"amount": 1.5, "kernel_size": (5, 5)},
        {"amount": 2.0, "kernel_size": (7, 7)},
        {"amount": 2.5, "kernel_size": (9, 9)}
    ]
}

In [ ]:
# Run hyperparameter tuning
tuned_results = hyperparameter_tuning(total_images, best_techniques_list, technique_param_grids, advanced_evaluation)
print("\nTuned Results:")

for step_name, result in tuned_results.items():
    print(f"{step_name}: Best Parameters: {result['Best Parameters']}, Best Score: {result['Best Score']}")

#### Interpretations of Results

Let's break down and interpret the hyperparameter tuning results for each step. Here's what each section tells us:

##### **1. Noise Reduction: Median Blur**
- **Best Parameters**: `{'ksize': 3}`
- **Best Score**: `-0.03`

**Interpretation**:
- **Median Blur** was selected as the best noise reduction technique.
- The slightly negative score (`-0.03`) indicates a small deviation from the original characteristics, with `ksize=3` performing the best among the tested values. This result shows that a smaller kernel size effectively reduces noise while avoiding excessive smoothing, which helps retain image clarity. Since noise negatively impacts image quality, preserving details while reducing noise was key to achieving a good balance.

##### **2. Histogram Equalization: Histogram Equalization**
- **Best Parameters**: `None`
- **Best Score**: `-inf`

**Interpretation**:
- The hyperparameter tuning for **Histogram Equalization** did not find any beneficial parameters, resulting in a score of `-inf`. This suggests that histogram equalization might not be effective for enhancing this specific dataset. Given that histogram equalization aims to enhance contrast, the lack of improvement implies that the initial contrast levels may already have been optimal, or that equalization introduced inconsistencies that were detrimental to the image quality.

##### **3. Binarization: Adaptive Threshold**
- **Best Parameters**: `{'adaptiveMethod': 1, 'blockSize': 11, 'C': 2}`
- **Best Score**: `78.84`

**Interpretation**:
- The **Adaptive Threshold** method, using the **Gaussian adaptive method** (`adaptiveMethod=1`), a **block size** of `11`, and a **constant C** of `2`, achieved a high score of `78.84`.
- This positive score indicates effective enhancement of key metrics such as contrast and sharpness, as well as noise reduction. The use of a **Gaussian adaptive method** along with a smaller block size allowed for better handling of local variations in illumination, which improved segmentation and detail clarity. The evaluation rewarded the technique because contrast was notably improved, which is crucial for extracting meaningful features in the images.

##### **4. Morphological Operations: Dilation**
- **Best Parameters**: `{'kernel_size': (9, 9)}`
- **Best Score**: `-0.01`

**Interpretation**:
- **Dilation** was selected as the best morphological operation for this dataset.
- A **kernel size of `(9, 9)`** provided the least negative score (`-0.01`), indicating that the larger kernel size was successful in connecting fragmented elements, which improved the structure of features in the image. The score's slight negativity implies a trade-off, where some minor details were lost, but the benefit of connecting important features outweighed the drawbacks, especially in terms of preparing the images for content recognition.

##### **5. Edge Detection: Canny Edge**
- **Best Parameters**: `{'threshold1': 50, 'threshold2': 150}`
- **Best Score**: `12.23`

**Interpretation**:
- The **Canny Edge Detection** method, with **thresholds of `50` and `150`**, achieved a score of `12.23`.
- The positive score indicates that these threshold values effectively highlighted edges, which was beneficial for extracting structures such as table lines and text outlines. The evaluation rewarded the improvement in contrast and sharpness, which contributed to making the details clearer and reducing the impact of noise. Lower thresholds allowed the detection of more edges, which in turn enhanced the structure and features of the image.

##### **Summary & Key Insights**:

1. **Negative Scores**:
   - Negative scores (e.g., **Noise Reduction** and **Morphological Operations**) indicate that the processed images deviated slightly from the original characteristics in ways that were detrimental based on the evaluation metrics.
   - The least negative scores represent the best parameters that resulted in minimal detrimental changes while preserving key features.

2. **Positive Scores**:
   - Positive scores (e.g., **Binarization** and **Edge Detection**) indicate significant improvements in metrics such as **contrast**, **sharpness**, and **detection of elements**.
   - Techniques with higher positive scores effectively enhanced image quality by improving features that are crucial for content recognition and extraction.

3. **Histogram Equalization Issue**:
   - The score of `-inf` for **Histogram Equalization** indicates that this technique was not effective for improving the evaluated characteristics. This could imply that the dataset's initial contrast was already optimal, or that the equalization process introduced artifacts that degraded the overall quality.

4. **Best Techniques Overview**:
   - For each step, the parameter combination with the highest score (least negative or most positive) was selected.
   - It is important to note that the magnitude of scores can vary greatly, depending on the evaluation metrics and their relative weights. In this evaluation, **contrast** and **sharpness** were given higher importance, leading to higher scores for techniques that excelled in these areas.

##### Recommendations:
- **Median Blur (Noise Reduction)**: A smaller kernel size (`ksize=3`) provided the least negative impact, suggesting that moderate noise reduction without excessive smoothing was most effective.
- **Adaptive Threshold (Binarization)**: Using a **Gaussian adaptive method** with a **smaller block size** resulted in the highest improvement in contrast and sharpness, which are crucial for effective segmentation.
- **Dilation (Morphological Operations)**: A larger kernel size (`9x9`) helped enhance the structure of the features, which was beneficial for preparing the images for subsequent recognition steps.
- **Canny Edge Detection**: Lower thresholds (`50` and `150`) proved effective for enhancing edge details, resulting in a high score and indicating that comprehensive edge detection was valuable for downstream tasks.

Overall, the results are consistent with expectations—lower parameters (e.g., less aggressive filtering or morphological operations) often preserve original characteristics better, while edge detection benefits from more comprehensive edge capturing with lower thresholds.
